In [1]:
import pandas as pd
import numpy as np
import os
from multiprocessing import Pool, cpu_count
from tqdm import tqdm_notebook as tqdm

In [84]:
def nan_score(df):
    return np.mean(np.mean(pd.notna(df)))


def nan_sanitization(args):
    """Drop iteratively columns and rows at step of <step> percent until the dataset has ration (non null values)/(all values) >= <percentage>"""
    path, df = args
    iteration = 0
    percentage = 0.70
    step = 0.01
    while nan_score(df) < percentage:
        if (iteration % 2) == 0:
            df = df.drop(columns=df.columns[np.mean(pd.notna(df)) < step])
        else:
            df = df.drop(df.index[np.mean(pd.notna(df), axis=1) < step])
        iteration += 1
        step *= 1.01

    df.to_csv(path)

In [85]:
dataframes = [("../sanitized_csv/{csv}".format(path=path, csv=csv),
               pd.read_csv(
                   "{path}/{csv}".format(path=path, csv=csv),
                   index_col="name"))
              for path, dirs, csvs in os.walk("../csv/") for csv in csvs
              if not csv.startswith(".")]

In [86]:
with Pool(cpu_count()) as p:
    list(tqdm(p.imap(nan_sanitization, dataframes), total = len(dataframes)))